In [1]:
import os
import autogen
from textwrap import dedent
import sys
from typing import Annotated

sys.path.append(os.path.abspath(".."))  # Add the parent directory
from finrobot.toolkits import register_toolkits
from autogen import AssistantAgent, UserProxyAgent
from autogen.cache import Cache
import requests
import json
import pandas as pd

In [2]:
work_dir = "./MultiAgent_work"
os.makedirs(work_dir, exist_ok=True)

In [3]:
llm_config_instant=[{
        "model": "llama-3.1-8b-instant",
        "api_type": "groq",
        "api_key": "API-KEY"
    }]

llm_config_gemma=[{
        "model": "gemma2-9b-it",
        "api_type": "groq",
        "api_key": "API-KEY" 
    }]

In [4]:
weather_analyst = AssistantAgent(
    name="analyzer",
    llm_config={"config_list": llm_config_gemma, "temperature": 0},
    system_message=dedent("""
        Your are a Weather data Analyst specifically for temperature and solar radiation data.
        Your task is to analyze the temperature and shortwave radiation weather data.
        Reply "TERMINATE" in the end when everything is done.
        """)
)
data_fetching_agent = AssistantAgent(
    name="Data_Provider",
    llm_config={"config_list": llm_config_gemma, "temperature": 0},
    system_message=dedent("""
        You are a weather data processing agent. Your task is to fetch the weather data of one month and process it using the provided function only.
        Your only task is to write exact string output of the provided function (no interpretation or comments).
        which ends with the word "TERMINATE".    
        Do not summarize, do not repeat the prompt, and do not add any explanation.
        """),
    max_consecutive_auto_reply=1,
)
user_proxy = UserProxyAgent(
    name="User_proxy",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and x.get(
        "content", "").endswith("TERMINATE"),
    max_consecutive_auto_reply=1,
    code_execution_config={
        "work_dir": work_dir,
        "use_docker": False
    }, 
)

In [5]:
def get_monthly_temperature(
    latitude: Annotated[float, "Latitude of location"], 
    longitude: Annotated[float, "Longitude of the location"], 
    start_date: Annotated[str, "Start date in format YYYY-MM-DD"], 
    end_date: Annotated[str, "End date in format YYYY-MM-DD"],
    save_path: Annotated[str, "Folder Path to save the temperature data file"]
) -> str:
    """
    Fetches daily temperature and shortwave radiation data for a given location and date range,
    and returns them as a JSON string.

    Outputs the beneficial data as string which can be used to analyze temperature and radiation data.
    """
    try:
        os.makedirs(save_path, exist_ok=True)
        json_file_path = os.path.join(save_path, f"temperature_{start_date}_{end_date}.json")

        if os.path.exists(json_file_path):
            with open(json_file_path, 'r') as file:
                weather_data = json.load(file)
        else:
            url = "https://archive-api.open-meteo.com/v1/archive"
            params = {
                "latitude": latitude,
                "longitude": longitude,
                "daily": "temperature_2m_mean,shortwave_radiation_sum",
                "start_date": start_date,
                "end_date": end_date,
                "timezone": "auto"
            }
            response = requests.get(url, params=params)
            if response.status_code == 200:
                weather_data = response.json()
                with open(json_file_path, "w") as file:
                    json.dump(weather_data, file, indent=2)
            else:
                return f"API Error {response.status_code}: {response.text}"

        # Validate response
        if "daily" not in weather_data or not all(
            key in weather_data["daily"] 
            for key in ["time", "temperature_2m_mean", "shortwave_radiation_sum"]
        ):
            return "Invalid data format received from API."

        dates = weather_data['daily']['time']
        temp_data = weather_data['daily']['temperature_2m_mean']
        radiation_data = weather_data['daily']['shortwave_radiation_sum']

        # combined_data = [
        #     {
        #         "date": date,
        #         "temperature_2m_mean": temp,
        #         "shortwave_radiation_sum": radiation
        #     }
        #     for date, temp, radiation in zip(dates, temp_data, radiation_data)
        # ]

        # data_file_path = os.path.join(save_path,f"combined_data.json")

        # with open(data_file_path, "w") as file:
        #             json.dump(combined_data, file, indent=2)
        # print(f"Processed Temperature data file saved at {data_file_path}")

        df = pd.DataFrame({
                'date': pd.to_datetime(dates),
                'temp': temp_data,
                'radiation': radiation_data
            })
        # Assume df has date, temp, radiation columns
        df['month'] = df['date'].dt.strftime('%B')
        
        monthly_stats = df.groupby('month').agg(
            avg_temp=('temp', 'mean'),
            max_temperature=('temp', 'max'),
            min_temperature=('temp', 'min'),
            temp_range=('temp', lambda x: x.max() - x.min()),
            avg_radiation=('radiation', 'mean'),
            radiation_std=('radiation', 'std')
        ).reset_index()

        report_lines = []

        for _, row in monthly_stats.iterrows():
            line = (
                f"In {row['month']}, the average temperature was {row['avg_temp']:.1f}°C, "
                f"average solar radiation was {row['avg_radiation']:.1f} MJ/m², "
                f"maximum temperature reached {row['max_temperature']:.1f}°C, "
                f"minimum was {row['min_temperature']:.1f}°C, "
                f"temperature range was {row['temp_range']:.1f}°C, "
                f"and radiation variability (std) was {row['radiation_std']:.2f} MJ/m²."
            )
            report_lines.append(line)
        
        final_report = "\n".join(report_lines)

        final_report += "\nTERMINATE"
        return final_report

    except Exception as e:
        return f"Exception occurred: {str(e)}"

tools1 = [
    {
        "function": get_monthly_temperature,
        "name": "get_weather_data",
        "description": "retrieve weather information from the Open-Meteo API and returns consolidated information about temperature and radiation data."
    }
]
register_toolkits(tools1, data_fetching_agent, user_proxy)

In [7]:
# def estimate_solar_energy_output(
#     combined_data_path: Annotated[str, "Combined data file path, required value of type working_directory/combined_data.json"],
#     panel_area_m2: Annotated[float, "Area of the solar panel in square meters"],
#     panel_efficiency: Annotated[float, "Efficiency of the panel (default: 18%)"],  # Default to 18% efficiency
#     save_path: Annotated[str, "Folder Path to save the temperature data file"]
# ) -> str:
#     """
#     Estimate solar energy output (in Wh) per day based on shortwave radiation data.
#     Returns:
#     - List of dicts with date and estimated energy output in Wh
#     """

#     output_data = []
#     MJ_to_Wh = 277.778

#     json_file_path = os.path.join(save_path,combined_data_path)
#     if os.path.exists(json_file_path):
#         with open(json_file_path,'r') as file:
#             combined_data = json.load(file)
#     else:
#         return f"Error: No data file found"

#     for entry in combined_data:
#         radiation = entry.get("shortwave_radiation_sum", 0)  # in MJ/m²
#         estimated_output = radiation * panel_area_m2 * panel_efficiency * MJ_to_Wh  # in Wh
        
#         output_data.append({
#             "date": entry["date"],
#             "estimated_energy_output_Wh": round(estimated_output, 2),
#             "temperature_2m_mean": entry.get("temperature_2m_mean", None),
#             "shortwave_radiation_sum": radiation
#         })

#     return json.dumps(output_data, indent=2)

# tools2 = [
#     {
#         "function": estimate_solar_energy_output,
#         "name": "estimate_solar_energy_output",
#         "description": "Using the data, named combined_data.json, present in working directory, estimates the solar energy output per day and returns this data as string"
#     }
# ]
# register_toolkits(tools2, weather_analyst, user_proxy)

In [8]:
with Cache.disk() as cache:
    autogen.initiate_chats(
        [
            {
                "sender": user_proxy,
                "recipient": data_fetching_agent,
                "message": dedent(f""" 
                1. Use only the provided function to fetch weather data (mean temperature and shortwave radiation) 
                   for latitude 40.7128 and longitude -74.0060, between 2020-05-01 and 2020-06-30.
                   
                2. The function returns a string after processing the data. Do not explain or rephrase its output.
                
                Store all files in {work_dir} (if required).
                """),
                "max_turn":0,
                "summary_method": "last_msg"
            },
            {
                "sender": user_proxy,
                "recipient": weather_analyst,
                "message": dedent(f""" 
                With the temperature and radiation data provided as context. 
                Analyze the fluctions in temperature, shortwave radiation and give report in 500 words.
                Reply Terminate if task is done.
                """),
                "max_turns":1,  # max number of turns for the conversation
                "summary_method": "last_msg"
            }
        ]
    )


********************************************************************************
Starting a new chat....

********************************************************************************
User_proxy (to Data_Provider):


1. Use only the provided function to fetch weather data (mean temperature and shortwave radiation) 
   for latitude 40.7128 and longitude -74.0060, between 2020-05-01 and 2020-06-30.

2. The function returns a string after processing the data. Do not explain or rephrase its output.

Store all files in ./MultiAgent_work (if required).


--------------------------------------------------------------------------------
Data_Provider (to User_proxy):



D:\Anaconda3\envs\finrobot\lib\site-packages\autogen\oai\groq.py:279: UserWarning: Cost calculation not available for model gemma2-9b-it
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


***** Suggested tool call (call_krg3): get_weather_data *****
Arguments: 
{"end_date":"2020-06-30","latitude":40.7128,"longitude":-74.0060,"save_path":"./MultiAgent_work","start_date":"2020-05-01"}
*************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_weather_data...
Call ID: call_krg3
Input arguments: {'end_date': '2020-06-30', 'latitude': 40.7128, 'longitude': -74.006, 'save_path': './MultiAgent_work', 'start_date': '2020-05-01'}
User_proxy (to Data_Provider):

***** Response from calling tool (call_krg3) *****
In June, the average temperature was 21.7°C, average solar radiation was 23.0 MJ/m², maximum temperature reached 25.6°C, minimum was 14.8°C, temperature range was 10.8°C, and radiation variability (std) was 5.56 MJ/m².
In May, the average temperature was 14.1°C, average solar radiation was 21.8 MJ/m², maximum temperature reached 22.3°C, minimum was 4.7°C, 

D:\Anaconda3\envs\finrobot\lib\site-packages\autogen\oai\groq.py:279: UserWarning: Cost calculation not available for model gemma2-9b-it
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


## Temperature and Solar Radiation Analysis: May vs. June

This report analyzes the temperature and shortwave radiation data for May and June, highlighting key fluctuations and trends.

**Temperature:**

* **June:** Showed a significant increase in average temperature compared to May, rising from 14.1°C to 21.7°C. This represents a 7.6°C increase. The maximum temperature also increased, reaching 25.6°C in June compared to 22.3°C in May. However, the minimum temperature in June (14.8°C) was higher than the minimum in May (4.7°C). This indicates a more consistent and warmer temperature range in June.
* **May:** Experienced a wider temperature range, with a difference of 17.6°C between the maximum and minimum temperatures. This suggests greater daily fluctuations in temperature compared to June.

**Solar Radiation:**

* **June:** Averaged 23.0 MJ/m² of shortwave radiation, slightly higher than May's average of 21.8 MJ/m². 
* **Variability:** Both months exhibited relatively similar radiat